In [ ]:
import pandas as pd
from src import *
import glob

# Gopro QAQC

## Original Gopro

### Generate a dataframe of corresponding images and labels as they are

In [ ]:
# the original images and labels
def original_image_label_paths():
    imgs_dan = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Dan_Dataset\images\*.jpg")
    imgs_hard_substrat = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Hard_Substrate\images\*.JPG")
    imgs_soft_substrat = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Soft_Substrate\images\*.JPG")
    imgs_gopro = imgs_hard_substrat+imgs_soft_substrat + imgs_dan

    pnglbls_hard_substrat = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Hard_Substrate\png_labels\*.png")
    pngs_dan = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Dan_Dataset\png_labels\*.png")
    pnglbls_soft_substrat = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Soft_Substrate\png_labels\*.png")
    pnglbls = pnglbls_hard_substrat+pnglbls_soft_substrat + pngs_dan

    yolo_dan = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Dan_Dataset\YOLO_labels\*.txt")
    lbls_hard_substrat = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Hard_Substrate\Yolo_labels\*.txt")
    lbls_soft_substrat = glob.glob(r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Soft_Substrate\Yolo_labels\*.txt")
    lbls = lbls_hard_substrat+lbls_soft_substrat + yolo_dan

    imgs_lbls_gopro = pd.DataFrame(np.c_[imgs_gopro, pnglbls, lbls], columns=["image_path", "png_label_path", "label_path"]) #.sample(frac=1, random_state = 42)
    # imgs_lbls_gopro["substrate"] = imgs_lbls_gopro.image_path.str.extract(r'([A-Z][a-z]{3}_Substrate)')
    imgs_lbls_gopro = imgs_lbls_gopro.reset_index(drop=True)
    imgs_lbls_gopro['unique_filename'] = general.make_imgs_nms_unique(imgs_lbls_gopro.image_path)
    imgs_lbls_gopro['image_id'] = imgs_lbls_gopro.unique_filename.apply(lambda x: x.split(".")[0])
    assert imgs_lbls_gopro.shape == imgs_lbls_gopro.drop_duplicates(subset='image_id').shape
    # imgs_lbls_gopro.to_csv(r'datasets\GoPro_datasets\GLSC_GoPro\GoPro_img_lbl_pths_unique.csv')
    # imgs_lbls_gopro = pd.read_csv(r'datasets\GoPro_datasets\GLSC_GoPro\GoPro_img_lbl_pths_unique.csv', index_col=0)
    imgs_lbls_gopro["image_id"] = imgs_lbls_gopro.image_path.apply(lambda x: os.path.basename(x).split(".")[0])
    print(imgs_lbls_gopro.shape)
    return imgs_lbls_gopro
imgs_lbls_gopro = original_image_label_paths()
imgs_lbls_gopro[0:5]

In [ ]:
def dans_master_lbl_df():
    dans = imgs_lbls_gopro[imgs_lbls_gopro.unique_filename.str.contains("Dan")]
    dans_master_gopro_yolo_labels = general.make_master_lbl_df_gopro(dans.label_path)
    dans_master_gopro_yolo_labels = dans_master_gopro_yolo_labels.rename(columns={'cls': 'cls_l', "x":'x_l', "y":'y_l', "w":'w_l', "h":'h_l', 'fish_id': 'ground_truth_id'})
    assert dans_master_gopro_yolo_labels.shape = dans_master_gopro_yolo_labels.drop_duplicates(subset=['x_l','y_l','w_l', 'h_l'])
    return dans_master_gopro_yolo_labels
dans_master_gopro_yolo_labels = dans_master_lbl_df()
dans_master_gopro_yolo_labels.head(2)

In [ ]:
# Previous QAQC
def glsc_infer_results_qaqc():
    prev_qaqc_pths = glob.glob(r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\*_edit\yolov8x_GoPro_*_scores_annot.xlsx")
    qaqc_dfs = [pd.read_excel(df, index_col=0) for df in prev_qaqc_pths]
    qaqc_df = pd.concat(qaqc_dfs)
    prev_scor_pths = glob.glob(r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\*_edit\yolov8x_GoPro_*_scores.csv")
    score_dfs = [pd.read_csv(df, index_col=0) for df in prev_scor_pths]
    score_df = pd.concat(score_dfs)
    score_df = score_df.drop_duplicates(subset=['image_name','ground_truth_id', 'x_l', 'y_l', 'w_l', 'h_l'])
    assert len(qaqc_df.detect_id.unique()) == len(qaqc_df)
    assert len(score_df.detect_id.unique()) == len(score_df)
    
    qaqc_df = pd.merge(score_df[['image_name', 'detect_id', 'names_l', 'cls_l', 'x_l', 'y_l', 'w_l', 'h_l', 'im_h_l', 'im_w_l', 'ground_truth_id']], 
                        qaqc_df[["detect_id", "qaqc"]], on = "detect_id")
    assert qaqc_df.shape == qaqc_df.drop_duplicates(subset=['image_name', 'x_l', 'y_l', 'w_l', 'h_l']).shape
    qaqc_df["image_id"] = qaqc_df.image_name.apply(lambda x: x.split(".")[0])
    qaqc_df = pd.merge(qaqc_df, dans_master_gopro_yolo_labels, on=["image_id", "ground_truth_id", 'cls_l', 'x_l', 'y_l', 'w_l', 'h_l'], how = "outer")
    qaqc_df = pd.merge(qaqc_df, imgs_lbls_gopro, on="image_id")
    
    qaqc_df.qaqc = qaqc_df.qaqc.fillna("U")
    qaqc_df.qaqc = qaqc_df.qaqc.replace({1.0:"Y", 0.0:"N"})
    qaqc_df.cls_l = qaqc_df.cls_l.astype(int)
    return qaqc_df
# use this dataset to create the qaqc dataset (dans not yet qaqc'd)
# qaqc_df = glsc_infer_results_qaqc()

### Using the QAQC filter and moving to new filepath with unique filenames

In [ ]:
def gopro_qaqc_to_unique(qaqc_df):
    for image_path in qaqc_df.image_path.unique():
        df_sub = qaqc_df[qaqc_df.image_path == image_path]
        df_sub = df_sub[(df_sub.qaqc=="Y") | (df_sub.qaqc=="U")]
        if not df_sub.empty:
            lbl = df_sub[["cls_l", "x_l", "y_l", "w_l", "h_l"]]
            id = df_sub.unique_filename.unique()[0].split(".")[0]
            imdest = r"datasets\GoPro_datasets\GLSC_unique_qaqc\images"
            lbdest = r"datasets\GoPro_datasets\GLSC_unique_qaqc\labels"
            lbl.to_csv(os.path.join(lbdest,id+".txt"), index=None, sep=" ", header=None)
            shutil.copy2(image_path, os.path.join(imdest, id+".jpg"))
# gopro_qaqc_to_unique(qaqc_df)

## LMBS Gopro qaqc

In [ ]:
# Hillary Inference
## make dataframe of original image paths and unique image names
def lmbs_infer_results_qaqc():
    LMBS_Goby_img_pths = general.list_files(r"datasets\GoPro_datasets\LMBS_original", ".jpg")
    unique_names = general.make_imgs_nms_unique(LMBS_Goby_img_pths)
    LMBS_Goby_Photos_df = pd.DataFrame(np.c_[LMBS_Goby_img_pths, unique_names], columns=["image_path", "filename"])
    # combine inference results with qaqc checks and image paths
    infer_results = pd.read_csv(r"datasets\GoPro_datasets\LMBS_original\LMBS_conf0.05_QAQC\LMBS_Goby_Photos_conf0.05_Yolo_predictions.csv", index_col=0)
    infer_results['orig_filename'] = infer_results.filename.apply(lambda x: x.split("_")[-1])
    assert infer_results.shape == infer_results.drop_duplicates(subset=['filename', 'x', 'y', 'w', 'h']).shape
    qaqc_results = pd.read_excel(r"datasets\GoPro_datasets\LMBS_original\LMBS_conf0.05_QAQC\LMBS_Goby_Photos_conf0.05_Yolo_predictions_QAQC_Nov2024_TO_USGS.xlsx", index_col=0)
    qaqc_results = qaqc_results.rename(columns={"filename":"orig_filename", "detect_id":"plot_id", "CHECK":"qaqc"})
    qaqc_results['image_id'] = qaqc_results.orig_filename.apply(lambda x: x.split(".")[0])
    qaqc_results['path_id'] = qaqc_results.path.apply(lambda x: x.replace("/","_"))
    qaqc_results['plot_id'] = qaqc_results.plot_id.apply(lambda x: x.replace(" ","_"))
    qaqc_results['detect_id'] = qaqc_results.path_id + "_" + qaqc_results.image_id + "_" + qaqc_results.plot_id
    assert qaqc_results.shape == qaqc_results.drop_duplicates(subset="detect_id").shape
    infer_results = pd.merge(infer_results, qaqc_results[["detect_id", "qaqc"]], on = "detect_id")
    infer_results = pd.merge(infer_results, LMBS_Goby_Photos_df, on="filename")
    assert infer_results.shape == infer_results.drop_duplicates(subset=['filename', 'x', 'y', 'w', 'h']).shape
    return infer_results
#use this dataframe to create new dataset
# lmbs_infer_results_qaqc = lmbs_infer_results_qaqc()

### Using the QAQC filter and moving to new filepath with unique filenames

In [ ]:
def lmbs_gopro_qaqc_to_unique(lmbs_infer_results_qaqc):
    for image_path in lmbs_infer_results_qaqc.image_path.unique():
        df_sub = lmbs_infer_results_qaqc[lmbs_infer_results_qaqc.image_path == image_path]
        df_sub = df_sub[df_sub.qaqc=="Y"]
        if not df_sub.empty:
            lbl = df_sub[["cls", "x", "y", "w", "h"]]
            id = df_sub.filename.unique()[0].split(".")[0]
            imdest = r"datasets\GoPro_datasets\LBMS_unique_qaqc\images"
            lbdest = r"datasets\GoPro_datasets\LBMS_unique_qaqc\labels"
            lbl.to_csv(os.path.join(lbdest,id+".txt"), index=None, sep=" ", header=None)
            shutil.copy2(image_path, os.path.join(imdest, id+".jpg"))
lmbs_gopro_qaqc_to_unique(lmbs_infer_results_qaqc)

# Create Train Test Split

In [ ]:
# functions
def get_im_w(image_path):
    im = Image.open(image_path)
    w, h = im.size
    return w
def get_im_h(image_path):
    im = Image.open(image_path)
    w, h = im.size
    return h
lim_w = lambda f: get_im_w(f)
lim_h = lambda f: get_im_h(f)

In [ ]:
imgs_all = glob.glob(r"datasets\GoPro_datasets\GLSC_unique_qaqc\images\*.jpg") + glob.glob(r"datasets\GoPro_datasets\LBMS_unique_qaqc\images\*.jpg")
lbls_all = glob.glob(r"datasets\GoPro_datasets\GLSC_unique_qaqc\labels\*.txt") + glob.glob(r"datasets\GoPro_datasets\LBMS_unique_qaqc\labels\*.txt")
gopro_all = generate_splits(img_pth_lst = imgs_all, bbox_pths=lbls_all, mer_pths=None).return_merged()
gopro_all["imh"], gopro_all["imw"] = gopro_all.image_path.apply(lim_h), gopro_all.image_path.apply(lim_w)

In [ ]:
gopro_all.groupby(by=["imh","imw"]).count()

In [ ]:
def do_train_test_valid_split(df, train_split=0.6, valid_split=0.2):
    l = len(df)
    print("n samples:", l)
    train_df = df.sample(int(train_split*l), random_state=42)
    df = df.drop(train_df.index)
    valid_df = df.sample(int(valid_split*l), random_state=42)
    df = df.drop(valid_df.index)
    test_df = df
    X_train, y_train = train_df.image_path.values, train_df.bbox_path.values
    X_valid, y_valid = valid_df.image_path.values, valid_df.bbox_path.values
    X_test, y_test = test_df.image_path.values, test_df.bbox_path.values
    print(f"training, testing, validation, {X_train.shape[0]}, {X_test.shape[0]},{X_valid.shape[0]}")
    return X_train, y_train, X_valid, y_valid, X_test, y_test, train_df, valid_df, test_df
X_train, y_train, X_valid, y_valid, X_test, y_test, train_df, valid_df, test_df = do_train_test_valid_split(gopro_all, train_split=0.77, valid_split=0.115)

In [ ]:
## copy images and labels
import shutil
def cpy_lbls(set, dst):
    if not os.path.exists(dst): os.makedirs(dst)
    for item in set:
        shutil.copy2(item, dst)
root =  r"datasets\GoPro_datasets\GLSC_LMBS_dataset"
# cpy_lbls(y_train, os.path.join(root, "train\\labels"))
# cpy_lbls(y_valid, os.path.join(root, "validation\\labels"))
# cpy_lbls(y_test, os.path.join(root, "test\\labels"))
# cpy_lbls(X_train, os.path.join(root, "train\\images"))
# cpy_lbls(X_valid, os.path.join(root, "validation\\images"))
# cpy_lbls(X_test, os.path.join(root, "test\\images"))

### Batches for MakeSense.ai QAQC

In [ ]:
root = r"datasets\GoPro_datasets\GLSC_unique_qaqc"
def create_batch_gopro_qaqc(root):
    batch_folder = "QAQC_batches"
    images = sorted(glob.glob(os.path.join(root, "images", "*.jpg")))
    labels = sorted(glob.glob(os.path.join(root, "labels","*.txt")))
    assert len(images) == len(labels), "images labels do not match"

    def chunk_list(lst, chunk_size=100):
        # Using list comprehension to break the list into chunks
        return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]
    
    chunks_im = chunk_list(images)
    chunks_lb = chunk_list(labels)
    for idx, im_lst, lb_lst in zip(range(len(chunks_im)), chunks_im, chunks_lb):
        dirname = os.path.join(root, batch_folder, f"batch_{idx}")
        for imsrc, lbsrc in zip(im_lst, lb_lst):
            imdst = os.path.join(dirname, "images")
            lbdst = os.path.join(dirname, "labels")
            if not os.path.exists(imdst): os.makedirs(imdst)
            if not os.path.exists(lbdst): os.makedirs(lbdst)
            shutil.copy2(imsrc, imdst)
            shutil.copy2(lbsrc, lbdst)
        # after copies, make labels.txt files
        definition_lst = os.path.join(dirname, "labels.txt")
        lst = os.listdir(lbdst)
        general.write_list_txt(definition_lst, lst)
        shutil.copy(definition_lst, lbdst)
# create_batch_gopro_qaqc(root)
